In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Consumption Growth

In [2]:
import yfinance as yf

# consumption growth
stocks = ['CELH', 'CLX', 'EL', 'ELF', 'MMYT', 'ONON', 'SHAK', 'SOVO', 'STRT', 'WING']
start_date = '2022-12-30'
end_date = '2023-12-31'
data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']
daily_returns = data.pct_change().iloc[1:, :]

mus = daily_returns.mean()*252
cov = daily_returns.cov()

[*********************100%%**********************]  10 of 10 completed


In [3]:
#- How many assests to include in each portfolio
n_assets = 10
#-- How many portfolios to generate
n_portfolios = 5000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

np.random.seed(422)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [4]:
noshort = pd.read_csv("julia_result/consumption_growth_noshort.csv", sep=';')
noshort = noshort.dropna()

short = pd.read_csv("julia_result/consumption_growth_short.csv", sep=';')
short = short.dropna()

risk = noshort['SDp'].tolist()
ret = noshort['ERp'].tolist()

risk2 = short['SDp'].tolist()
ret2 = short['ERp'].tolist()

In [6]:
short

,SDp,ERp,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
1,0.011660,0.00,-0.0285,0.4666,0.2539,-0.1018,0.0840,-0.0479,0.0396,0.1393,0.1269,0.0679
2,0.011550,0.01,-0.0279,0.4620,0.2483,-0.0984,0.0862,-0.0470,0.0412,0.1398,0.1267,0.0691
3,0.011442,0.02,-0.0273,0.4575,0.2427,-0.0950,0.0884,-0.0461,0.0428,0.1402,0.1266,0.0702
4,0.011337,0.03,-0.0267,0.4529,0.2371,-0.0916,0.0906,-0.0451,0.0443,0.1407,0.1264,0.0713
5,0.011235,0.04,-0.0262,0.4484,0.2316,-0.0882,0.0928,-0.0442,0.0459,0.1411,0.1263,0.0724
...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.013082,0.76,0.0152,0.1206,-0.1703,0.1555,0.2525,0.0226,0.1605,0.1739,0.1157,0.1538
78,0.013217,0.77,0.0158,0.1161,-0.1759,0.1588,0.2547,0.0236,0.1621,0.1744,0.1155,0.1549
79,0.013354,0.78,0.0163,0.1115,-0.1815,0.1622,0.2569,0.0245,0.1637,0.1748,0.1154,0.1560
80,0.013493,0.79,0.0169,0.1070,-0.1870,0.1656,0.2591,0.0254,0.1653,0.1753,0.1152,0.1572


In [9]:
#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0.05 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
import plotly.graph_objects as go
import plotly.offline as py
data1 = go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                   y=mean_variance_pairs[:,0], 
                   #- Add color scale for sharpe ratio   
                   marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
#                                  line=dict(width=1),
                                  colorscale="earth"#,
 #                                 colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                   mode='markers',
                   name="Risky Portfolio")

data2 = go.Scatter(x=[0.012733],
                   y=[0.68],
                   marker=dict(color=['orangered'],
                              showscale=True,
                              size=10),
                   name="Optimal Risky Portfolio (Without Short)")

data3 = go.Scatter(x=[0.013493],
                   y=[0.8],
                   marker=dict(color=['orangered'],
                              showscale=True,
                              size=10),
                   name="Optimal Risky Portfolio (With Short)")

#- Add title/labels
layout = dict(template='plotly_white',
                  xaxis=dict(title='Risk (Volatility)'),
                  yaxis=dict(title='Return'),
                  #title='Minimal Variance Frontier of Growth Stocks - China',
                  legend=dict(yanchor="bottom", xanchor="left", y=0.99, x=0.01))
#                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

frontier = go.Scatter(x=risk,
                      y=ret,
                      mode="lines",
                      line=dict(color='black',
                                dash='dash'),
                      name="Minimum Variance Frontier without Short")

frontier2 = go.Scatter(x=risk2,
                       y=ret2,
                       mode="lines",
                       line=dict(color='darkviolet',
                                dash='dash'),
                       name="Minimum Variance Frontier with Short")

CAL = go.Scatter(x=[0, 0.012733],
                 y=[0.05, 0.68],
                 line=dict(color='orangered',
                           dash='solid'),
                 mode='lines',
                 name='Capital Allocation Line (Without Short)')

CAL2 = go.Scatter(x=[0, 0.013493],
                 y=[0.05, 0.8],
                 line=dict(color='orangered',
                           dash='dash'),
                 mode='lines',
                 name='Capital Allocation Line (With Short)')

fig=go.Figure(data=[data1,data2, data3,frontier, frontier2, CAL, CAL2],layout=layout)
py.iplot(fig)

## Consumption Value

In [104]:
import yfinance as yf

# consumption growth
stocks = ['CALM', 'CASY', 'COKE', 'DECK', 'F', 'LANC', 'LOPE', 'LRN', 'ULTA', 'YUMC']
start_date = '2022-12-30'
data = yf.download(stocks, start=start_date)['Adj Close']
daily_returns = data.pct_change().iloc[1:, :]

mus = daily_returns.mean()*252
cov = daily_returns.cov()

[*********************100%%**********************]  10 of 10 completed


In [105]:
noshort = pd.read_csv("julia_result/consumption_value_noshort.csv", sep=';')
noshort = noshort.dropna()

short = pd.read_csv("julia_result/consumption_value_short.csv", sep=';')
short = short.dropna()

risk = noshort['SDp'].tolist()
ret = noshort['ERp'].tolist()

risk2 = short['SDp'].tolist()
ret2 = short['ERp'].tolist()

In [106]:
#- How many assests to include in each portfolio
n_assets = 10
#-- How many portfolios to generate
n_portfolios = 5000

#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

np.random.seed(422)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [109]:
#-- Plot the risk vs. return of randomly generated portfolios
#- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0.05 #-- Include risk free rate here for sharpe ratio

#-- Create Plot
import plotly.graph_objects as go
import plotly.offline as py
data1 = go.Scatter(x=mean_variance_pairs[:,1]**0.5, 
                   y=mean_variance_pairs[:,0], 
                   #- Add color scale for sharpe ratio   
                   marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
#                                  line=dict(width=1),
                                  colorscale="RdBu"#,
 #                                 colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                   mode='markers',
                   name="Risky Portfolio")

data2 = go.Scatter(x=[0.01148],
                   y=[0.56],
                   marker=dict(color=['orangered'],
                              showscale=True,
                              size=10),
                   name="Optimal Risky Portfolio")
#- Add title/labels
layout = dict(template='plotly_white',
                  xaxis=dict(title='Risk (Volatility)'),
                  yaxis=dict(title='Return'),
                  #title='Minimal Variance Frontier of Growth Stocks - China',
                  legend=dict(yanchor="bottom", xanchor="left", y=0.99, x=0.01))
#                  coloraxis_colorbar=dict(title="Sharpe Ratio"))

frontier = go.Scatter(x=risk,
                      y=ret,
                      mode="lines",
                      line=dict(color='black',
                                dash='dash'),
                      name="Minimum Variance Frontier without Short")

frontier2 = go.Scatter(x=risk2,
                       y=ret2,
                       mode="lines",
                       line=dict(color='darkviolet',
                                dash='dash'),
                       name="Minimum Variance Frontier with Short")

CAL = go.Scatter(x=[0, 0.01148],
                 y=[0.05, 0.56],
                 line=dict(color='orangered',
                           dash='solid'),
                 mode='lines',
                 name='Capital Allocation Line (Without Short)')
fig=go.Figure(data=[data1,data2, frontier, frontier2, CAL],layout=layout)
py.iplot(fig)